#Generación de Texto

In [ ]:
import requests
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
import numpy as np
import os
import time
import urllib.request

Descargaremos el libro de  [The Great Gatsby, by F. Scott Fitzgerald](https://www.gutenberg.org/files/64317/64317-0.txt)

In [ ]:
path_to_file = tf.keras.utils.get_file('gatsby.txt', 'https://www.gutenberg.org/files/64317/64317-0.txt')

319488/306258 [===============================] - 0s 1us/step


In [ ]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [ ]:
print(f'Length of text: {len(text)} characters')

Length of text: 296704 characters


##Sneak peak del libro 

In [ ]:
print(text[945:5000])
text=text[945:]

The Great Gatsby
				  by
			 F. Scott Fitzgerald


                           Table of Contents

I
II
III
IV
V
VI
VII
VIII
IX


                              Once again
                                  to
                                 Zelda

  Then wear the gold hat, if that will move her;
  If you can bounce high, bounce for her too,
  Till she cry “Lover, gold-hatted, high-bouncing lover,
  I must have you!”

  Thomas Parke d’Invilliers


                                  I

In my younger and more vulnerable years my father gave me some advice
that I’ve been turning over in my mind ever since.

“Whenever you feel like criticizing anyone,” he told me, “just
remember that all the people in this world haven’t had the advantages
that you’ve had.”

He didn’t say any more, but we’ve always been unusually communicative
in a reserved way, and I understood that he meant a great deal more
than that. In consequence, I’m inclined to reserve all judgements, a
habit that has opened up many cu

### Ahora analicemos un poco de los caracteres que tiene este texto

In [ ]:
print(f'Este libro tiene: {len(text)} caracteres de los cuales ')
vocab = sorted(set(text))
print(f'{len(vocab)} son únicos')

Este libro tiene: 295759 caracteres de los cuales 
94 son únicos


#Empecemos a vectorizar nuestro texto para su análisis

* comenzaremos usando la función de `preprocessing.StringLookup` puede convertir cada carácter en un ID numérico. Sólo necesita que el texto haya sido dividido en tokens primero.

[Link documentación](https://www.tensorflow.org/api_docs/python/tf/keras/layers/StringLookup)

In [ ]:
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab), mask_token=None)


In [ ]:
example_texts = ['abcdefg', 'xyz', 'ABC', 'XYZ', ",*."]

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
print('print 1:::: chars ' ,chars ,end='\n')

print('xxxxx')
print('------------------------------------------------------------------------------------------------------------------xxxxx')
print('xxxxx')
print
ids = ids_from_chars(chars)
print('print 2:::: ids ', ids)

print 1:::: chars  <tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z'],
 [b'A', b'B', b'C'], [b'X', b'Y', b'Z'], [b',', b'*', b'.']]>
xxxxx
------------------------------------------------------------------------------------------------------------------xxxxx
xxxxx
print 2:::: ids  <tf.RaggedTensor [[58, 59, 60, 61, 62, 63, 64], [81, 82, 83], [30, 31, 32], [53, 54, 55],
 [13, 12, 15]]>


Dado que el objetivo de este notebook es generar texto leíble por humanos, también será importante invertir esta representación y recuperar cadenas de forma legibles. 
* Para ello puede utilizar `preprocessing.StringLookup(..., invert=True)`. 

[Link documentación](https://www.tensorflow.org/api_docs/python/tf/keras/layers/StringLookup)

Aquí en lugar de pasar el vocabulario original generado con `sorted(set(text))` se utiliza el método `get_vocabulary()` de la capa `preprocessing.StringLookup` para que los tokens `[UNK]` se establezcan de la misma manera.

In [ ]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z'],
 [b'A', b'B', b'C'], [b'X', b'Y', b'Z'], [b',', b'*', b'.']]>

Puedes `tf.strings.reduce_join` para volver a unir los caracteres en cadenas. 

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz', b'ABC', b'XYZ', b',*.'], dtype=object)

#Tarea del modelo

Dado un carácter, o una secuencia de caracteres, ¿cuál es el siguiente carácter más probable? Esta es la tarea para la que se entrena el modelo. La entrada del modelo será una secuencia de caracteres, y se entrena el modelo para predecir la salida, el siguiente carácter, en cada paso de tiempo.

![imagen](https://i.ytimg.com/vi/VAMKuRAh2nc/maxresdefault.jpg)

A continuación, divide el texto en secuencias de ejemplo. Cada secuencia de entrada contendrá `seq_length` del texto.

Para cada secuencia de entrada, los objetivos correspondientes contienen la misma longitud de texto, excepto desplazada un carácter a la derecha.

Por lo tanto, divide el texto en trozos de `seq_length+1`. Por ejemplo, digamos que `seq_length` es 4 y nuestro texto es "Hola". La secuencia de entrada sería "Hol", y la secuencia de destino "ola".

Para ello, primero se utiliza la función `tf.data.Dataset.from_tensor_slices` para convertir el vector de texto en un flujo de índices de caracteres.


In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(295759,), dtype=int64, numpy=array([49, 65, 62, ..., 15,  3,  2])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
ids_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

In [ ]:
for ids in ids_dataset.take(100):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

T
h
e
 
G
r
e
a
t
 
G
a
t
s
b
y



	
	
	
	
 
 
b
y



	
	
	
 
F
.
 
S
c
o
t
t
 
F
i
t
z
g
e
r
a
l
d









 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
T
a
b
l
e
 
o
f
 
C
o
n
t
e
n
t


In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)


El método `batch` permite convertir fácilmente estos caracteres individuales en secuencias del tamaño deseado.
A continuación, divide el texto en secuencias de ejemplo. Cada secuencia de entrada contendrá `seq_length` del texto.


In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
print('xxxxxx')
print('print objeto sequences 1 :::' , type(sequences))
print('xxxxxx')
print('-----------------------------------------------------------------------------xxxxxx')

for seq in sequences.take(1):
  print(chars_from_ids(seq))

xxxxxx
print objeto sequences 1 ::: <class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>
xxxxxx
-----------------------------------------------------------------------------xxxxxx
tf.Tensor(
[b'T' b'h' b'e' b' ' b'G' b'r' b'e' b'a' b't' b' ' b'G' b'a' b't' b's'
 b'b' b'y' b'\r' b'\n' b'\t' b'\t' b'\t' b'\t' b' ' b' ' b'b' b'y' b'\r'
 b'\n' b'\t' b'\t' b'\t' b' ' b'F' b'.' b' ' b'S' b'c' b'o' b't' b't' b' '
 b'F' b'i' b't' b'z' b'g' b'e' b'r' b'a' b'l' b'd' b'\r' b'\n' b'\r' b'\n'
 b'\r' b'\n' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' '
 b' ' b'T' b'a' b'b' b'l' b'e' b' ' b'o' b'f' b' ' b'C' b'o' b'n' b't'
 b'e' b'n' b't' b's'], shape=(101,), dtype=string)


Es más fácil ver lo que hace esto si se unen los tokens de nuevo en cadenas:

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'The Great Gatsby\r\n\t\t\t\t  by\r\n\t\t\t F. Scott Fitzgerald\r\n\r\n\r\n                           Table of Contents'
b'\r\n\r\nI\r\nII\r\nIII\r\nIV\r\nV\r\nVI\r\nVII\r\nVIII\r\nIX\r\n\r\n\r\n                              Once again\r\n             '
b'                     to\r\n                                 Zelda\r\n\r\n  Then wear the gold hat, if that '
b'will move her;\r\n  If you can bounce high, bounce for her too,\r\n  Till she cry \xe2\x80\x9cLover, gold-hatted, hi'
b'gh-bouncing lover,\r\n  I must have you!\xe2\x80\x9d\r\n\r\n  Thomas Parke d\xe2\x80\x99Invilliers\r\n\r\n\r\n                         '


Para el entrenamiento necesitarás un conjunto de datos de pares `(input, label)`. Donde `input` y 
son secuencias. En cada paso de tiempo la entrada es el carácter actual y la etiqueta es el siguiente carácter. 

Aquí hay una función que toma una secuencia como entrada, la duplica y la desplaza para alinear la entrada y la etiqueta para cada paso de tiempo:

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text
#ejemplo de esta función
split_input_target(list("Tensorflow"))


(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'The Great Gatsby\r\n\t\t\t\t  by\r\n\t\t\t F. Scott Fitzgerald\r\n\r\n\r\n                           Table of Content'
Target: b'he Great Gatsby\r\n\t\t\t\t  by\r\n\t\t\t F. Scott Fitzgerald\r\n\r\n\r\n                           Table of Contents'


### Crear lotes de entrenamiento

Ya usamos `tf.data` para dividir el texto en secuencias manejables. Pero antes de introducir estos datos en el modelo, es necesario barajar los datos y empaquetarlos en lotes.

In [ ]:
# Batch size
BATCH_SIZE = 64

# Tamaño del buffer para barajar el conjunto de datos 
#(TF data está diseñado para trabajar con secuencias posiblemente infinitas, 
#por lo que no intenta barajar toda la secuencia en memoria. En su lugar, 
#mantiene un buffer en el que baraja los elementos).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

 # Ahora sí, el modelo

En esta sección vamos a definir el modelo como una subclase de `keras.Model` (Para más detalles: [Creación de nuevas capas y modelos mediante subclases](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

Este modelo tiene tres capas:

* `tf.keras.layers.Embedding`: La capa de entrada. Una tabla de búsqueda entrenable que asignará cada carácter-ID a un vector con dimensiones `embedding_dim`;
* `tf.keras.layers.GRU`: Un tipo de RNN con tamaño `units=rnn_units` (También puedes usar una capa LSTM aquí).
* `tf.keras.layers.Dense`: La capa de salida, con salidas `vocab_size`. Produce un logit para cada carácter del vocabulario. Estos son la probabilidad logarítmica de cada carácter según el modelo.


In [ ]:
# tamaño del vocabulario de caracteres 
vocab_size = len(vocab)

# dimesión de embedding
embedding_dim = 256

# Número de unidades de RNN 
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    #Capas de mi red
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)
    ##


  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x
      

In [ ]:
model = MyModel(
    # Asegúrese de que el tamaño del vocabulario coincide con las capas de `StringLookup`.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

Para cada carácter, el modelo busca el embedding, ejecuta la red un paso de tiempo con el embedding como entrada, y aplica la capa densa para generar logits que predicen la log-verosimilitud del siguiente carácter:
![imagen](https://www.tensorflow.org/text/tutorials/images/text_generation_training.png)

#Ahora a probar el modelo

In [ ]:
dataset.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")



(64, 100, 95) # (batch_size, sequence_length, vocab_size)


In [ ]:
# correr primero celdas más abajo de dataset take y el ciclo for 
input_example_batch

<tf.Tensor: shape=(64, 100), dtype=int64, numpy=
array([[65, 62,  4, ..., 58, 77, 62],
       [60, 65,  3, ...,  4, 80, 65],
       [ 4, 80, 58, ..., 62, 71, 69],
       ...,
       [58, 77, 62, ..., 66, 71, 64],
       [72, 71, 76, ..., 14, 14, 14],
       [91, 77,  4, ..., 62,  4, 63]])>

In [ ]:
target_example_batch


<tf.Tensor: shape=(64, 100), dtype=int64, numpy=
array([[62,  4, 65, ..., 77, 62, 70],
       [65,  3,  2, ..., 80, 65, 62],
       [80, 58, 75, ..., 71, 69, 82],
       ...,
       [77, 62, 75, ..., 71, 64,  4],
       [71, 76, 89, ..., 14, 14, 14],
       [77,  4, 65, ...,  4, 63, 72]])>

In [ ]:
model(input_example_batch)

<tf.Tensor: shape=(64, 100, 95), dtype=float32, numpy=
array([[[ 4.06068191e-03,  1.73543103e-03,  3.46108852e-03, ...,
          3.48487496e-03, -6.97220021e-05, -1.55786157e-03],
        [-7.24973461e-06, -6.56283635e-04, -8.21116101e-03, ...,
         -5.07999398e-03,  1.03581036e-02, -7.60579016e-03],
        [-9.05201398e-03, -9.01258271e-03, -8.88958480e-03, ...,
         -3.44724604e-03,  1.08368639e-02,  1.94353983e-03],
        ...,
        [-1.51887571e-03,  2.00958527e-03, -2.72662519e-03, ...,
          2.12261151e-03,  9.12176445e-03, -7.94811547e-03],
        [-7.18055433e-03,  7.51613188e-05, -5.14623430e-03, ...,
          1.28297973e-02,  5.52483648e-03, -8.99291970e-03],
        [-6.44281739e-03,  2.16606911e-03, -1.22641036e-02, ...,
          2.76850117e-03,  1.37801385e-02, -1.09999692e-02]],

       [[ 2.35853135e-03,  4.14716778e-03,  1.19151874e-02, ...,
         -1.78833921e-02,  4.97799320e-03,  7.53703760e-04],
        [ 4.98188986e-03,  4.73816600e-03,  9.19

In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  24320     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  97375     
                                                                 
Total params: 4,059,999
Trainable params: 4,059,999
Non-trainable params: 0
_________________________________________________________________


Para obtener las predicciones reales del modelo es necesario muestrear de la distribución de salida, para obtener los índices de caracteres reales. Esta distribución está definida por los logits sobre el vocabulario de caracteres.

Nota: Es importante  _sample_ a partir de esta distribución, ya que si se toma el _argmax_ de la distribución, el modelo puede entrar fácilmente en un bucle.

Veamos para el primer ejemplo del lote:



In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

Esto nos da, en cada paso de tiempo, una predicción del siguiente índice de caracteres:

In [ ]:
sampled_indices

array([78, 35, 59, 93, 79, 59, 76, 91, 23, 50,  6, 29, 80, 10, 13,  7,  1,
       39, 53, 35, 63, 87, 85, 29,  5,  3, 19, 86,  1, 53, 40, 72, 35, 26,
        4, 28, 87, 12, 14,  0, 77, 39, 71, 86, 37, 94, 39, 59, 50, 60, 87,
       35, 56, 74, 48, 14, 83, 32, 49, 24, 68, 46, 35, 36, 13, 75, 79, 49,
       15, 69, 78, 53, 21, 46, 15, 86, 26, 58, 69, 26, 37, 75, 72,  2, 90,
       24, 39, 48, 26, 89, 72, 68, 18, 46, 21, 19, 93, 11, 49, 33])

Descodifícalos para ver el texto predicho por este modelo no entrenado:

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'he had anything to say.\r\n\r\n\xe2\x80\x9cAnything to say about what?\xe2\x80\x9d inquired Gatsby politely.\r\n\r\n\xe2\x80\x9cWhy\xe2\x80\x94any state'

Next Char Predictions:
 b'uFb\xe2\x80\x9dvbs\xe2\x80\x996U"?w(,$\tJXFf\xc3\xb4\xc3\xa9?!\r2\xc3\xaa\tXKoF9 ;\xc3\xb4*-[UNK]tJn\xc3\xaaH\xe2\x80\xa6JbUc\xc3\xb4F[qS-zCT7kQFG,rvT.luX4Q.\xc3\xaa9al9Hro\n\xe2\x80\x987JS9\xe2\x80\x94ok1Q42\xe2\x80\x9d)TD'


#A entrenar nuestro modelo

Para obtener las predicciones reales del modelo es necesario muestrear de la distribución de salida, para obtener los índices de caracteres reales. Dado el estado anterior de la RNN, y la entrada en este paso de tiempo, predecir la clase del siguiente carácter.

También introduciremos funciones pérdida y un optimizador

La función de pérdida estándar `tf.keras.losses.sparse_categorical_crossentropy` funciona en este caso porque se aplica en la última dimensión de las predicciones.

Debido a que su modelo devuelve logits, necesita establecer la bandera `from_logits`.

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 95)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.554342


Un modelo recién inicializado no debería estar muy seguro de sí mismo, los logits de salida deberían tener todos magnitudes similares. Para confirmar esto se puede comprobar que la exponencial de la pérdida media es aproximadamente igual al tamaño del vocabulario. Una pérdida mucho mayor significa que el modelo está seguro de sus respuestas erróneas, y que está mal inicializado:

In [ ]:
tf.exp(mean_loss).numpy()

95.044174

Configuramos el procedimiento de entrenamiento utilizando el método `tf.keras.Model.compile.` Y también utilizamos `tf.keras.optimizers.Adam` con los argumentos por defecto y la función de pérdida.

In [ ]:
model.compile(optimizer='adam', loss=loss)

#Configuramos los chekpoints

Utiliza un `tf.keras.callbacks.ModelCheckpoint` para asegurar que los puntos de control se guardan durante el entrenamiento:

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

##Ahora sí, ejecutamos nuestro entrenamiento

In [ ]:
EPOCHS = 15
#lo dejamos en poquitos para que nuestro notebook no tarde años jaja 

![waiting](https://img.devrant.com/devrant/rant/r_2392200_uXijd.jpg)
![waiting](https://media.makeameme.org/created/still-waiting-for-577ae62ee8.jpg)

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/15
45/45 [==============================] - 9s 140ms/step - loss: 3.5741
Epoch 2/15
45/45 [==============================] - 7s 145ms/step - loss: 2.5706
Epoch 3/15
45/45 [==============================] - 7s 138ms/step - loss: 2.3331
Epoch 4/15
45/45 [==============================] - 7s 139ms/step - loss: 2.2228
Epoch 5/15
45/45 [==============================] - 7s 137ms/step - loss: 2.1114
Epoch 6/15
45/45 [==============================] - 7s 137ms/step - loss: 1.9866
Epoch 7/15
45/45 [==============================] - 7s 137ms/step - loss: 1.8783
Epoch 8/15
45/45 [==============================] - 7s 138ms/step - loss: 1.7811
Epoch 9/15
45/45 [==============================] - 7s 137ms/step - loss: 1.6898
Epoch 10/15
45/45 [==============================] - 7s 137ms/step - loss: 1.6047
Epoch 11/15
45/45 [==============================] - 7s 137ms/step - loss: 1.5283
Epoch 12/15
45/45 [==============================] - 7s 136ms/step - loss: 1.4577
Epoch 13/15
45/45 [======

##Ahora a generar texto

La forma más sencilla de generar texto con este modelo es ejecutarlo en un bucle, y seguir el estado interno del modelo mientras se ejecuta.

Para generar texto, la salida del modelo se devuelve a la entrada

Cada vez que se llama al modelo se le pasa un texto y un estado interno. El modelo devuelve una predicción para el siguiente carácter y su nuevo estado. Vuelve a pasar la predicción y el estado para seguir generando texto.
A continuación se hace una predicción de un solo paso:

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Ahora lo vamos a ejecutar en un bucle para generar un texto. Al observar el texto generado, verás que el modelo sabe cuándo poner mayúsculas, hacer párrafos e imitar un vocabulario de escritura similar al de Fitzgerald. Con el pequeño número de epochs de entrenamiento, aún no ha aprendido a formar frases coherentes.

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Gatsby found out he was dreaming'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Gatsby found out he was dreaming looks waved sick up in a fladyations
work that me should beet with ever a few up
somebody toplarg mertanicr addie as Chicago, with the
know. “You’r began to alm.”

OHro out of a will humarly taking twition was voice Nownstanded through the saw that
they met he was sevanded them togerhers floor. She
susposed without fants away and leaned wo kind orce of contanis girls. To years out scoor.

Py Gatsby had breatfly train. A Cawer—Their heart stirdly giel stations from though
him up in this Mancen wither pershanes curfon elormen supportusion of a hint was
still, underst nearing retuminquils over is though gleeces of
a grovel questions that it who mag been a motter
of tho sing was the Frive, everythings.”


 I rame dose spulled intentions more. Supperted Jay Gatsby, breaking and
puole of chairs and I cuternished him up, how!) paid
durking by Neire, where id each other past, his ore
hongaser. Af him that I
drove or the tolp reassure grow, histerw her fig to Ga

#Challenge:

Ahora te toca a ti experimentar con este notebook. Genera una copia donde elijas otro libro que analizar y vamos a mejorar los resultados. 

Lo más fácil que puedes hacer para mejorar los resultados es entrenarlo durante más tiempo 

(prueba con `EPOCHS = 15`).

También puedes experimentar con una cadena de inicio diferente, intentar añadir otra capa de RNN para mejorar la precisión del modelo.  

La principal prueba será cambiar la capa *self.gru = tf.keras.layers.GRU* por una capa *tf.keras.layers.LSTM*

https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM